## Week 2 Class activities
This notebook is a starting point for the exercises and activities that we'll do in class.

Before you attempt any of these activities, make sure to watch the video lectures for this week.

### Scraping permit data
Here's the code that we saw in the video lecture that queries the City of Seattle permit website, gets a dataframe of permits (including the URL), and then digs down further into that permit-specific URL.

In [1]:
# get the permit data from the API
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = 'https://data.seattle.gov/resource/ht3q-kdvx.json' # copied and pasted from the webpage
r = requests.get(url)
df = pd.DataFrame(json.loads(r.text))

df = df.head(5) # get the first 5 rows, so we don't overload the city's website.

# get an example link
permiturl = df.loc[0,'link']['url']
print(permiturl)

# request that page and get the soup object
r = requests.get(permiturl)
soup = BeautifulSoup(r.text)
print(soup.prettify())

https://cosaccela.seattle.gov/portal/customize/LinkToRecord.aspx?altId=3001212-LU
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="en-US" ng-app="appAca" xml:lang="en-US" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head id="ctl00_Head1">
  <link href="../App_Themes/Default/_progressbar.css" rel="stylesheet" type="text/css"/>
  <link href="../App_Themes/Default/breadcrumb.css" rel="stylesheet" type="text/css"/>
  <link href="../App_Themes/Default/Calendar.css" rel="stylesheet" type="text/css"/>
  <link href="../App_Themes/Default/custom.css" rel="stylesheet" type="text/css"/>
  <link href="../App_Themes/Default/font.css" rel="stylesheet" type="text/css"/>
  <link href="../App_Themes/Default/form.css" rel="stylesheet" type="text/css"/>
  <link href="../App_Themes/Default/grid.css" rel="stylesheet" type="text/css"/>
  <link href="../App

In [2]:
# then we wrote this code to extract the project description 
links = soup.find_all('td')
for link in links:
    if 'Project Description' in link.text: 
        sublinks = link.find_all('td')
        description = sublinks[1].text
        # once we find a description, we exit
        break
    
print(description)

PROJECT CANCELLED 12/8/2010 -- This short plat has an ECA exemption in the project planning template. A limited exemption was granted. Processing short plat with the ECA exemption #3002070.


<div class="alert alert-block alert-info">
<strong>Exercise:</strong> If you look at the example, there is a <strong>Legal Description</strong> section. Extract that to a variable and print it.
</div>

In [64]:
# your code here
# do some detective work to figure out where Legal Description is contained
links = soup.find_all('td')
for link in links:
    if 'Legal Description' in link.text: 
        sublinks = link.find_all('td')
        description = sublinks[3].text
        # once we find a description, we exit
        break
    
print(description)



Development Site Parcel:DV0001932Legal Description:PAR A, LBA #8806752, THT POR BLK 13, KINNEARS 1ST RAINIER BEACH ADD, AND POR TRC A, BLK 2, GUTHERIES TERRACE PARK, AND POR SE 1/4 (FILE)




In [100]:
# your code here
links = soup.find_all('td')
for link in links:
    if 'Legal Description' in link.text: 
        sublinks = link.find_all('td')
        description = sublinks[3].text
        # once we find a description, we exit
        break
    
print(sublinks[3])

<td class="MoreDetail_BlockContent">
<div id="ctl00_PlaceHolderMain_PermitDetailList1_palParceList">
<div class="MoreDetail_ItemCol MoreDetail_ItemCol1"><h2>Development Site Parcel:</h2><div class="ACA_SmLabel ACA_SmLabel_FontSize">DV0001932</div></div><div class="MoreDetail_ItemCol MoreDetail_ItemCol2"><h2>Legal Description:</h2><div class="ACA_SmLabel ACA_SmLabel_FontSize">PAR A, LBA #8806752, THT POR BLK 13, KINNEARS 1ST RAINIER BEACH ADD, AND POR TRC A, BLK 2, GUTHERIES TERRACE PARK, AND POR SE 1/4 (FILE)</div></div>
</div>
</td>


In [110]:
# your code here
# do some detective work to figure out where Legal Description is contained
def getDescription(urldict):
    permiturl = urldict['url']
    r = requests.get(permiturl)
    soup = BeautifulSoup(r.text)
    links = soup.find_all('td')
    for link in links:
        if 'Legal Description' in link.text: 
            sublinks = link.find_all('td')
            description = sublinks[3].text
            # once we find a description, we return it and exit the function
            return description 
    
    return '' # if we don't find it, return an empty string

# Now let's apply this function to the first link in our dataframe
urldict = df.loc[0,'link']
getDescription(urldict)

# create a copy of the first 10 rows of the dataframe.
smalldf = df.head(10).copy()  

# for each row in smallDf, we pass the link column to getDescription
descriptions = smalldf['link'].apply(getDescription)

# what's the description object? It's a pandas Series (basically, a one-column DataFrame)
print(type(descriptions))
print(descriptions)

<class 'pandas.core.series.Series'>
0    \n\nDevelopment Site Parcel:DV0001932Legal Des...
1    \n\n\nREQUIRED DOCUMENTS\n\n\nMust this be rec...
2    \n\n\nREQUIRED DOCUMENTS\n\n\nMust this be rec...
3    \n\nDevelopment Site Parcel:DV0009583Legal Des...
4    \n\n\nLAND USE NOTICES\n\n\nNotice Type:Applic...
Name: link, dtype: object


In [111]:
# So we can insert that into the dataframe as a new column
smalldf['legal description'] = descriptions
# we could have done this in one step: 
# smalldf['newdescription'] = smalldf['link'].apply(getDescription) 
smalldf

,permitnum,permitclass,permitclassmapped,permittypemapped,description,statuscurrent,relatededg_landusepermit,originaladdress1,originalcity,originalstate,...,housingunitsremoved,housingunitsadded,applieddate,issueddate,expiresdate,decisiondate,permittypedesc,contractorcompanyname,estprojectcost,legal description
0,3001212-LU,Single Family/Duplex,Residential,Master Use Permit,PROJECT CANCELLED 12/8/2010 -- This short plat...,Canceled,"{'type': 'Point', 'coordinates': [-122.2517206...",6519 S BANGOR ST,SEATTLE,WA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\n\nDevelopment Site Parcel:DV0001932Legal Des...
1,3001271-LU,Single Family/Duplex,Residential,Master Use Permit,Land Use Permit to adjust the boundary between...,Completed,"{'type': 'Point', 'coordinates': [-122.3569286...",4226 1ST AVE NW,SEATTLE,WA,...,0.0,0.0,2005-12-16,2006-05-15,2007-11-15,2006-05-10,NaN,NaN,NaN,\n\n\nREQUIRED DOCUMENTS\n\n\nMust this be rec...
2,3001310-LU,Single Family/Duplex,Residential,Master Use Permit,Land use application to adjust the boundary be...,Completed,"{'type': 'Point', 'coordinates': [-122.3026217...",941 23RD AVE S,SEATTLE,WA,...,NaN,NaN,2007-02-14,2008-08-28,2011-08-14,2008-08-13,NaN,NaN,NaN,\n\n\nREQUIRED DOCUMENTS\n\n\nMust this be rec...
3,3001312-LU,N/A,N/A,Master Use Permit,Cancelled due to no activity for more than 9 y...,Canceled,"{'type': 'Point', 'coordinates': [-122.2913013...",3131 E MADISON ST,SEATTLE,WA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\n\nDevelopment Site Parcel:DV0009583Legal Des...
4,3001440-LU,Commercial,Non-Residential,Master Use Permit,PROJECT CANCELLED 5/23/2011 -- Project On Hold...,Canceled,"{'type': 'Point', 'coordinates': [-122.3721853...",9030 13TH AVE NW,SEATTLE,WA,...,NaN,NaN,2005-08-12,NaN,NaN,NaN,NaN,NaN,NaN,\n\n\nLAND USE NOTICES\n\n\nNotice Type:Applic...


<div class="alert alert-block alert-info">
<strong>Exercise:</strong> Now turn that into a function that you can apply to each row of your dataframe. Add a new column, <strong>legal_description</strong>, to your dataframe.
</div>

### Fixing errors
We'll do more scraping in just a moment. But first, let's do some examples of how to interpret an error message, and fix it.

<div class="alert alert-block alert-info">
<strong>Exercise:</strong> Each of the cells below will generate an error. Look at the error message and see if you can figure out how to fix it. (Don't Google it until you try to figure it out based on the error message.)
</div>

In [114]:
# the housingunitsremoved and housingunitsadded give useful information
# let's create a new column with netunits
df['netunits'] = df.housingunitsadded.astype(float) - df.housingunitsremoved.astype(float)

In [115]:
df['netunits']

0    NaN
1    0.0
2    NaN
3    NaN
4    NaN
Name: netunits, dtype: float64

In [128]:
# print the address of the first row
print('Address of first row is {}. Permit type is '.format(df.iloc[1].originaladdress1))

Address of first row is 4226 1ST AVE NW. Permit type is 


In [131]:
# Convert the number of housing units to integers
# and then summarize

df['unitsadded_numeric'] = df.housingunitsadded.astype(float)
df.unitsadded_numeric.describe()

count    1.0
mean     0.0
std      NaN
min      0.0
25%      0.0
50%      0.0
75%      0.0
max      0.0
Name: unitsadded_numeric, dtype: float64

### Scraping craigslist

In the lecture, we saw how to scrape the main page (the list of posts).

What if you want to get more information about (say) a particular apartment?

Here's the code from the lecture that gets a dataframe of the first 120 posts. Notice that there is a `url` column.

In [ ]:
url = 'https://losangeles.craigslist.org/search/lac/hhh'
r = requests.get(url)

soup = BeautifulSoup(r.content)
posts = soup.find_all('li', class_= 'result-row')

postList = []

for post in posts:
    result_price = post.find('span', class_= 'result-price')
    if result_price is None:
        price = None
    else:
        price = result_price.text
    
    resulthood = post.find('span', class_= 'result-hood')
    if resulthood is None:
        neighborhood = None
    else:
        neighborhood = resulthood.text 
        
    # we can also have our if..else statements as a one-liner
    # this is identical to the above
    neighborhood = None if resulthood is None else resulthood.text

    housing = post.find('span', class_= 'housing')
    housingsize = None if housing is None else housing.text
        
    # these two fields seem to be always present, so no need to check for None
    title = post.find('a', class_= 'result-title').text
    url = post.find('a', class_= 'result-title')['href']

    # now put them in the dictionary, and append to our list
    postList.append({'price': price, 'neighborhood':neighborhood, 
                     'housingsize':housingsize, 'title':title, 'url':url})

df = pd.DataFrame(postList)
df.head()

<div class="alert alert-block alert-info">
<strong>Exercise:</strong> For the first url in your dataframe, use requests to get the content of the post. (No need to create a soup object yet.)
</div>

In [ ]:
# your code here
# put the output of the request in a variable called r
# so you can access the content like this
print(r.content)

Now let's extract more information from the page. We have a couple of strategies here. First, we could skip trying to parse the page with `BeautifulSoup`, and just see if particular bits of text are present.

For example, what transportation modes does the post emphasize? Do they mention Section 8 vouchers? Some of this might be exploratory—we can see what type of language is included, and then parse in a more structured way (e.g. distinguishing between "No Section 8" and "Section 8 welcome").

<div class="alert alert-block alert-info">
    <strong>Exercise:</strong> Write a function that will return True if Section 8 is mentioned, otherwise False.

*Hint*: the `in` operator is a simple way to do this. For example:

In [ ]:
'plan' in 'urban planning'

In [ ]:
'plan' in 'Urban Planning' 

In [ ]:
# your code here to return Section 8 information

Most of the post is free-form text. So there's not going to be much value added by `BeautifulSoup`.

The exceptions are (i) parking, and (ii) the geographic coordinates.

<div class="alert alert-block alert-info">
    <strong>Exercise:</strong> Write a function that will return True if the apartment has no parking, and also returns the lat/lon of the apartment

*Hint*: First, create a `soup` object. Then, look and see what tag and class encloses this information. Then, you can experiment with `find` and `find_all` with this tag and class.

In [ ]:
# your code here

<div class="alert alert-block alert-info">
<strong>Exercise:</strong> Apply this function to your dataframe, and create new columns for parking, lat, and lon.
</div>

In [ ]:
# your code here

<div class="alert alert-block alert-info">
<h3>What you should have learned</h3>
<ul>
  <li>Gain confidence in experimenting with code - exploring different objects, writing functions, and so on</li>
  <li>Learn how to extract information from a scraped webpage - how to do the detective work.</li>
  <li>Gain confidence in debugging errors.</li>
</ul>
</div>